In [2]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

In [40]:
RDATE = 20220630

In [41]:
#/*** RSME Base ***/

RSME_BASE  = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\07_RSME\\"+str(RDATE)+"\RSME"+str(RDATE)+".xlsx")
RSME_BASE.columns = RSME_BASE.columns.str.upper()


In [5]:
CUST_ISCPARENT = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\Python\User defined\cust_iscparent.xlsx')

In [6]:
Econ_Sector_Dsc = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\02_Econ_Sector_Dsc\Econ_Sector_Dsc.txt", sep = "\t", header = 0, low_memory = False)
Econ_Sector_Dsc.columns = Econ_Sector_Dsc.columns.str.replace(" ", "_")

In [7]:
Econ_Sector_Dsc.columns = Econ_Sector_Dsc.columns.str.upper()

In [9]:
#Teradata cust
location_cust = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\02_Teradata\01_Credit_Risk_Reporting\01_Cust\\"+str(RDATE)+"\\S001_004_ACRM_CUST"+str(RDATE)+".txt"
cust = pd.read_csv(location_cust, sep = "|", header = 0, error_bad_lines = False)
cust.columns = cust.columns.str.replace(" ", "_")
cust.M_SUB_SUB_MARKET_SEGMENT.replace({'    ': -9999}, inplace=True)

In [42]:
RSME_BASE1 = RSME_BASE[['ACCOUNT_NO','ECONOMIC_SECTOR_CD','ECON_SECTOR','M_BNM_BALANCE','CUS_NO']]
RSME_BASE.ECONOMIC_SECTOR_CD = RSME_BASE.ECONOMIC_SECTOR_CD.astype(str)

Econ_Sector_Dsc1 = Econ_Sector_Dsc[['ECON_SECTOR_CD','GEN1_DESCRIPTION']]
Econ_Sector_Dsc.ECON_SECTOR_CD = Econ_Sector_Dsc.ECON_SECTOR_CD.astype(str)

CUST_ISCPARENT1 = CUST_ISCPARENT[['ISC_CODE','ISC_PARENT']]
CUST_ISCPARENT1['ISC_CODE1'] = CUST_ISCPARENT1.ISC_CODE
CUST_ISCPARENT1.ISC_CODE = CUST_ISCPARENT.ISC_CODE.astype(str)

cust1 = cust[['CUS_NO','NAME','INDUSTRIAL_SECTOR']]


In [43]:
RSME_ECONISC3 = RSME_BASE1.merge(cust1,on='CUS_NO',how='left')

RSME_ECONISC3.INDUSTRIAL_SECTOR = RSME_ECONISC3.INDUSTRIAL_SECTOR.map(str)
RSME_ECONISC3.INDUSTRIAL_SECTOR = RSME_ECONISC3.INDUSTRIAL_SECTOR.str.strip()
RSME_ECONISC3.INDUSTRIAL_SECTOR = RSME_ECONISC3.INDUSTRIAL_SECTOR.str.upper()
CUST_ISCPARENT1.ISC_CODE = CUST_ISCPARENT1.ISC_CODE.map(str)
CUST_ISCPARENT1.ISC_CODE = CUST_ISCPARENT1.ISC_CODE.str.strip()
CUST_ISCPARENT1.ISC_CODE = CUST_ISCPARENT1.ISC_CODE.str.upper()
RSME_ECONISC3_1 = RSME_ECONISC3.merge(CUST_ISCPARENT1.rename(columns={'ISC_CODE':'INDUSTRIAL_SECTOR'}).drop_duplicates(),on='INDUSTRIAL_SECTOR',how='left')

RSME_ECONISC3_1.ECONOMIC_SECTOR_CD = RSME_ECONISC3_1.ECONOMIC_SECTOR_CD.map(str)
RSME_ECONISC3_1.ECONOMIC_SECTOR_CD = RSME_ECONISC3_1.ECONOMIC_SECTOR_CD.str.strip()
RSME_ECONISC3_1.ECONOMIC_SECTOR_CD = RSME_ECONISC3_1.ECONOMIC_SECTOR_CD.str.upper()
Econ_Sector_Dsc1.ECON_SECTOR_CD = Econ_Sector_Dsc1.ECON_SECTOR_CD.map(str)
Econ_Sector_Dsc1.ECON_SECTOR_CD = Econ_Sector_Dsc1.ECON_SECTOR_CD.str.strip()
Econ_Sector_Dsc1.ECON_SECTOR_CD = Econ_Sector_Dsc1.ECON_SECTOR_CD.str.upper()
RSME_ECONISC3_2 = RSME_ECONISC3_1.merge(Econ_Sector_Dsc1.rename(columns={'ECON_SECTOR_CD':'ECONOMIC_SECTOR_CD'}),on='ECONOMIC_SECTOR_CD',how='left')

RSME_ECONISC3_2.ISC_PARENT.fillna('23.Not Populated',inplace=True)

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [44]:
RSME_ECONISC3_3 = RSME_ECONISC3_2[['ACCOUNT_NO','ECONOMIC_SECTOR_CD','ECON_SECTOR','GEN1_DESCRIPTION','ISC_PARENT','M_BNM_BALANCE','NAME','CUS_NO']]


In [13]:
RPSIA = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\17_Sustainability_ESG high risk sectors\20220331\RPSIA Fikri\RPSIA_Mar22.xlsx", sheet_name='Sheet1')


In [45]:
RPSIA1 = RPSIA.fillna(0)#.groupby(['GCIF'])[['Outstanding']].sum().reset_index()


RPSIA1.GCIF = RPSIA1.GCIF.astype(str)
RPSIA1.GCIF = RPSIA1.GCIF.str.strip()
RPSIA1.GCIF = RPSIA1.GCIF.str.upper()

RSME_ECONISC3_3.CUS_NO = RSME_ECONISC3_3.CUS_NO.astype(str)
RSME_ECONISC3_3.CUS_NO = RSME_ECONISC3_3.CUS_NO.str.strip()
RSME_ECONISC3_3.CUS_NO = RSME_ECONISC3_3.CUS_NO.str.upper()

ex_RPSIA = RSME_ECONISC3_3.merge(RPSIA1[['GCIF']].drop_duplicates('GCIF').rename(columns={'GCIF':'CUS_NO'}), on='CUS_NO', how='left', indicator='True')

ex_RPSIA_FINAL = ex_RPSIA.iloc[np.where(ex_RPSIA['True']!='both')]

In [46]:
SME_RPSIA = ex_RPSIA.iloc[np.where(ex_RPSIA['True']=='both')]
SME_RPSIA

,ACCOUNT_NO,ECONOMIC_SECTOR_CD,ECON_SECTOR,GEN1_DESCRIPTION,ISC_PARENT,M_BNM_BALANCE,NAME,CUS_NO,True


In [47]:
ex_RPSIA_FINAL['GEN1_DESCRIPTION'].fillna('NA', inplace=True)
ex_RPSIA_FINAL['NAME'].fillna('NA', inplace=True)
ex_RPSIA_FINAL['ISC_PARENT'].fillna('NA', inplace=True)
ex_RPSIA_FINAL['M_BNM_BALANCE'].fillna(0, inplace=True)

a = ex_RPSIA_FINAL.groupby(['GEN1_DESCRIPTION','NAME'])[['M_BNM_BALANCE']].sum().reset_index()
b = ex_RPSIA_FINAL.groupby(['ISC_PARENT','NAME'])[['M_BNM_BALANCE']].sum().reset_index()

In [48]:
TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(RDATE)[:4]+'\\'+str(RDATE)[:6]+'\working\Total_Impaired_Final_'+str(RDATE)+'.txt',sep = ",", header = 0, low_memory = False)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.upper()
TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER = TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)

In [49]:
TOTAL_IMPAIRED_FINAL.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL.CUSTOMER_ID.astype(str)
TOTAL_IMPAIRED_FINAL.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL.CUSTOMER_ID.str.strip()
TOTAL_IMPAIRED_FINAL.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL.CUSTOMER_ID.str.upper()

ex_TOTAL_IMPAIRED_FINAL3 = TOTAL_IMPAIRED_FINAL.merge(RPSIA1[['GCIF']].drop_duplicates('GCIF').rename(columns={'GCIF':'CUSTOMER_ID'}), on='CUSTOMER_ID', how='left', indicator='True')



ex_TOTAL_IMPAIRED_FINAL3_FINAL = ex_TOTAL_IMPAIRED_FINAL3.iloc[np.where(ex_TOTAL_IMPAIRED_FINAL3['True']!='both')]

In [50]:
IF_RPSIA = ex_TOTAL_IMPAIRED_FINAL3.iloc[np.where(ex_TOTAL_IMPAIRED_FINAL3['True']=='both')]
IF_RPSIA

,N_RUN_SKEY,V_ENTITY_CODE,N_ACCT_SKEY,V_ACCOUNT_NUMBER,V_ORIGINAL_ACCOUNT_NUMBER,CUSTOMER_ID,V_D_CUST_FIRST_NAME,F_EXPOSURE_DEFAULT_STATUS_FLAG,F_IMPAIRMENT_STATUS,F_IA_IND,PREV_STAGE_IFRS,CURR_STAGE_IFRS,V_CORPORATE_REMEDIAL_CD,MKT_SUB_SEGMENT,V_PROD_GROUP_DESC,PROD_LV1,PROD_LV2,PROD_LV3,PROD_LV4,PROD_LV5,PROD_LV6,V_ACCOUNT_CLASSIFICATION_CODE,OUTSTANDING,EIR,AMORT_COST,CARRYING_AMT,DRAWN_AMT,UNDRAWN_AMT,EAD_BEFORE_MOD,MOD_GAIN_LOSS,REMAINING_MOD_GAIN_LOSS,EAD_AFTER_MOD,ECL_OPENING_ORI_CCY,ECL_OPENING_BAL_REVAL,ECL_OPENING_1ST_JAN,ECL_CHARGED,ECL_WRITE_BACK,ECL_WRITE_OFF,ECL_FX_DIFF,ECL_CLOSING_BAL,NET_ECL_PL,INDIVIDUAL_ALLOWANCE,DCF_AMT,UWI_OPENING_ORI_CCY,UWI_OPENING_BAL_REVAL,UWI_OPENING_1ST_JAN,UWI_CHARGED,UWI_WRITE_BACK,UWI_WRITE_OFF,UWI_FX_DIFF,UWI_CLOSING_BAL,UWI_FTM,WRITE_OFF_AMOUNT_RCY,WRITE_OFF_YTD,WRITE_OFF_AMOUNT,EXPOSURE_LIMIT,LEDGER_BAL,TOTAL_12M_ECL_BFORE_FL,TOTAL_12M_ECL_AFTER_FL,TOTAL_LIFETIME_BFORE_FL,TOTAL_LIFETIME_AFTER_FL,TOTAL_ECL_BFORE_FL,TOTAL_ECL_AFTER_FL,DEFAULT_RULE,COUNT_RECORDS,V_GL_CODE,CURR_RATING,ORIG_RATING,CURR_RATING_BAND_CATEGORY,CURR_RATING_BAND,ECL_METHOD,RS_INCLUSION,V_CCY_CODE,V_RELEASE_TYPE_CD,F_RPSIA_FLAG,SMF_FLAG,V_SEGMENT_ID,N_MONTH_IN_ARREARS,N_DELINQUENT_DAYS,V_LEVEL5_LOB_CODE,MIS_BY_PURPOSE,V_D_CUST_INDUSTRY,MIS_BY_NOB,V_PROD_CODE,V_PROD_DESC,V_PROD_FAMILY_DESC,MIS_BY_CONCEPT,V_CLASS_CODE,V_D_CUST_TYPE,MIS_BY_CUSTOMER_TYPE,V_PURPOSE_DESC,V_BUSINESS_UNIT_CODE,V_BUSINESS_UNIT_DESC,V_REGION_CODE,V_ACCT_BRANCH_CODE,V_REGION_DESC,V_D_ACCOUNT_STATUS,N_RCY_CONVERSION_FACTOR,N_TWELVE_MONTHS_PD,N_LIFETIME_PD,N_LGD_PERCENT,V_BASEL_LGD_POOL_ID,N_EFFECTIVE_INTEREST_RATE,N_CURR_INTEREST_RATE,V_INTEREST_METHOD,N_INTEREST_FREQ,V_INTEREST_FREQ_UNIT,SUM_A_N_INT_INCOME_,SUM_A_N_INT_INCOME_YTD_RCY_,F_AKPK_FLAG,D_AKPK_DATE,D_FINAL_DISBURSEMENT_DATE,D_ISSUE_DATE,D_LAST_PAYMENT_DATE,D_LAST_DELINQUENT_DATE,D_LATEST_DELINQUENT_DATE,F_MODIFICATION_FLAG,D_MODIFICATION_DATE,F_COMMITMENT_FLAG,F_SMA_FLAG,WL_FLAG,N_CCF_PERCENT,N_DRAWN_CCF_PERCENT,SUM_A_N_SANCTION_AMOUNT_,N_DISCOUNTING_FACTOR,F_EXP_REVOLVING_FLAG,F_EMP_ACCT_FLAG,F_NEW_ACCT_FLG,F_DISBURSED_IND,F_RESCHEDULE_IND,D_RESCHEDULE_DATE,F_RESTRUCTURED_IND,D_RESTRUCTURED_DATE,F_REMOVE_RESCHEDULE_IND,F_REMOVE_RESTRUCTURE_IND,F_REPACKAGING_IND,D_REPACKAGE_DATE,F_REPRICE_FLAG,D_REPRICE_DATE,CORPORATE_SCORECARD,V_SECTOR_CATEGORY,V_PD_TERM_STRUCTURE_ID,V_LGD_TERM_STRUCTURE_ID,N_ACCOUNT_TENOR,F_BANKRUPTCY_FLAG,F_REMOVE_BANKRUPTCY_FLAG,MATURITY_DATE,COLLATERAL_BAND,NPL_STATUS,F_PREF_TREATMENT_FLAG,GIL,V_STAGE_REASON_DESC,GIL_DEC_21,STATUS,CHECKING,ECL_WRITE_OFF___FINAL_,STATUS_WOFF,NO,LAST_MONTH,TRUE_FALSE,GIL_APR21,VAR__FTM,UNNAMED__162,OUTSTANDING_AFTER_EIR,OUTSTANDING_AFTER_EIR_UWI,F_RPSIA,F_NCIF_CUST,F_NCIF_ACCT,NOB_GROUP,NOB_GROUP_DESC,PRODUCT_GROUP,NOB_GROUP2,NOB_GROUP_DESC2,NPF_TAG,INDUSTRIAL_SECTOR,ISC_PARENT,True
0,19156,MIB,3379790,139415546,764016811075,19492758,TRURICH RESOURCES SDN BHD,Y,Y,0.0,0.0,3,N,CORPORATE BANKING,LOANS_OTH,On Balance Sheet,Assets,Loans And Advances,Term Loans,Others,TERM FINANCING-I,AMRTCOST,4.03e+08,0.0,4.03e+08,7.52e+07,3.52e+08,0.0,4.03e+08,280714.37,280714.37,4.04e+08,6.72e+07,2.96e+08,2.80e+08,0.0,3.18e+06,0.0,1.58e+07,2.93e+08,-3.18e+06,3.28e+08,7.57e+07,6.55e+06,2.89e+07,2.73e+07,6.19e+06,0.0,0.0,1.54e+06,3.51e+07,1.43e+06,0.0,0.0,0.0,3.53e+08,4.03e+08,0.0,0,1.72e+08,1.72e+08,2.93e+08,3.28e+08,N,1,120301210114,R23,R13,MAYBANK RATING,NPL,Specific Provision Methodology,0,USD,0.0,N,N,MY_Non-Retail,30.0,901.0,1120.0,WORKING CAPITAL,1113,"AGRICULTURE,HUNTING AND FORESTRY",H1_OTHLNS,TERM FINANCING-I,H1,MURABAHAH,121,201,Domestic Business Enterprise -Others,0.0,26471,BR KL MAIN OFFICE,R08,6401,FEDERAL TERRITORY,Active,0.23,1.0,1,0.42,0,5.24,5.24,0.0,0.0,0.0,330324.1,1.74e+06,N,01-JAN-00,22-AUG-17,14-AUG-17,0.0,0,0.0,Y,31-MAR-19,N,0,0,0.0,0.0,0.0,0.0,N,N,0,0,N,0,N,0,0,0,Y,08-FEB-19,N,01-JAN-00,OH,07,0.0,MY_Non-Retail_CB_U,59,N,0.0,14-AUG-27,UNSECURED,NPL THRESHOLD NOT EXCEEDED,0.0,3.69e+08,Impairment Status Y,3.48e+08,Amount r

In [51]:
ex_TOTAL_IMPAIRED_FINAL3_FINAL['MIS_BY_PURPOSE'].fillna('NA', inplace=True)
ex_TOTAL_IMPAIRED_FINAL3_FINAL['ISC_PARENT'].fillna('NA', inplace=True)
ex_TOTAL_IMPAIRED_FINAL3_FINAL['V_D_CUST_FIRST_NAME'].fillna('NA', inplace=True)
ex_TOTAL_IMPAIRED_FINAL3_FINAL['OUTSTANDING'].fillna(0, inplace=True)
ex_TOTAL_IMPAIRED_FINAL3_FINAL['OUTSTANDING_AFTER_EIR_UWI'].fillna(0, inplace=True)

TOTAL_IMPAIRED_FINAL1 = ex_TOTAL_IMPAIRED_FINAL3_FINAL.iloc[np.where(ex_TOTAL_IMPAIRED_FINAL3_FINAL.PRODUCT_GROUP=='05.SME')].groupby(['MIS_BY_PURPOSE','ISC_PARENT','V_D_CUST_FIRST_NAME'])[['OUTSTANDING','OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

D:\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [52]:
writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\202208_RAM\Corp & SME Q15 & Q16\RAM_SME_'+str(RDATE)+'(exclude rpsia).xlsx',engine='xlsxwriter')

ex_RPSIA_FINAL.to_excel(writer2,sheet_name='RSME_ECONISC3 (SME - working)' ,index = False)
ex_TOTAL_IMPAIRED_FINAL3_FINAL.to_excel(writer2,sheet_name='RSME_IFISCPUR (IF - working)' ,index = False)
a.to_excel(writer2,sheet_name='GEN1_DESCRIPTION' ,index = False)
b.to_excel(writer2,sheet_name='ISC_PARENT' ,index = False)
TOTAL_IMPAIRED_FINAL1.to_excel(writer2,sheet_name='RSME_IFISCPUR' ,index = False)
IF_RPSIA.to_excel(writer2,sheet_name='SME IF - RPSIA' ,index = False)
SME_RPSIA.to_excel(writer2,sheet_name='SME - RPSIA' ,index = False)
writer2.save()